In [12]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import re
import time
import csv

import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
import math
from nltk.tokenize import word_tokenize # or use some other tokenizer

I got the directions on how to do this webscraping from:
https://medium.freecodecamp.org/how-to-scrape-websites-with-python-and-beautifulsoup-5946935d93fe

I'm web scraping the comments from WebMD on the drug Gilenya.  The rating is the Effectivness rating.  They also have a Ease of Use and Satisfaction rating.

In [13]:
quote_page1 = 'https://www.webmd.com/drugs/drugreview-64439-Abilify-oral.aspx?drugid=64439&drugname=Abilify-oral&pageIndex='
quote_page2 = '&sortby=3&conditionFilter=-500'

In [14]:
#num_pages = 500
num_pages = 4
my_list=[]

for i in range(num_pages):
    #print(i)
    url = quote_page1 + str(i) + quote_page2
    #print(url)
    #page = urlopen(url)
    headers = {'User-Agent':'Mozilla/5.0'}
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    reviews = soup.find_all('div', attrs={'class': 'userPost'})
    
    for r in reviews:
        com = r.find('p', id=re.compile("^comFull*"))
        if com is not None:
            com = com.text.replace('Comment:','').replace('Hide Full Comment','')
            rate = int(r.find_all('span', attrs={'class': 'current-rating'})[0].text.replace('Current Rating:','').strip())
            rate = rate + int(r.find_all('span', attrs={'class': 'current-rating'})[1].text.replace('Current Rating:','').strip())
            rate = rate + int(r.find_all('span', attrs={'class': 'current-rating'})[2].text.replace('Current Rating:','').strip())
            rate = int(rate)/3.0
            my_list.append({'comment': com, 'rating': rate})
    #time.sleep(1)
len(my_list)

20

In [15]:
my_list[5]

{'comment': 'This medication (as an add on to Citalopram) makes me want to get up early every day and be productive.  I feel happier than I ever thought possible with the addition of this med (having dealt with minor chronic depression for most of my adult life).',
 'rating': 185.0}

Now that we have all the comments we need to save them to a csv file.

In [5]:
keys = my_list[0].keys()
with open('gilenya_effectivness.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(my_list)